In [103]:
from aswe.utils.request import http_request
from datetime import date
import os

In [6]:
headers = {
  'x-rapidapi-key': os.getenv("Sports_API_key"),
  'x-rapidapi-host': 'v1.basketball.api-sports.io'
}

In [11]:
def get_NBA_standings() -> list:
    data = http_request(f'https://v1.basketball.api-sports.io/standings?league=12&season=2022-2023', headers=headers).json()
    
    wc_standings = []
    ec_standings = []
    conferences = []
    for team in data["response"][0]:
        if team["group"]["name"] == "Eastern Conference" or team["group"]["name"] == "Western Conference": 
            if team["group"]["name"] not in conferences:
                conferences.append(team["group"]["name"])
                print(conferences[-1])
            if team["group"]["name"] == "Eastern Conference":
                ec_standings.append(f"{team['position']}. {team['team']['name']} wins: {team['games']['win']['total']} losses: {team['games']['lose']['total']}")
            if team["group"]["name"] == "Western Conference":
                wc_standings.append(f"{team['position']}. {team['team']['name']} wins: {team['games']['win']['total']} losses: {team['games']['lose']['total']}")
    return [wc_standings, ec_standings]

In [64]:
def get_team_id(team_name: str) -> int:
    team_name = team_name.replace(' ', '%20')
    data = http_request(f'https://v1.basketball.api-sports.io/teams?name={team_name}', headers=headers).json()
    
    if data["response"] == []:
        return None
    
    return data["response"][0]["id"]

In [100]:
def get_team_game_today(team_name: str) -> list:
    today = date.today()
    team_name = team_name.replace(' ', '%20')
    id = get_team_id(team_name)
    if id == None:
        return None
    data = http_request(f'https://v1.basketball.api-sports.io/games?date={today}&timezone=Europe/Berlin&league=12&season=2022-2023&team={id}', headers=headers).json()
    
    if data["response"] == []:
        return None
    
    games = []
    for game in data["response"]:
        games.append(f"{game['teams']['home']['name']} {game['scores']['home']['total']} - {game['scores']['away']['total']} {game['teams']['away']['name']}")
    return games